In [ ]:
#Information, Inference Networks: Tutorial 5.  S.C., R.M., F.Z.
#Analysis of Protein Sequence Data to infer protein structure 
#Contact Prediction from the Multiple Sequence Alignments of the Trypsin Hynibitor Protein (PF00014)
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import numpy.matlib
import numpy.linalg as LA
from numpy.linalg import inv
np.set_printoptions(threshold=np.inf)

In [ ]:
def letter2number(a): 
    #to convert the a.a. letters into integer numbers from 0 to 20
    switcher = {
        '-': 0,
        'A': 1,
        'C': 2,
        'D':3,
        'E':4,
        'F':5,
        'G':6,
        'H':7,
        'I':8,
        'K':9,
        'L':10,
        'M':11,
        'N':12,
        'P':13,
        'Q':14,
        'R':15,
        'S':16,
        'T':17,
        'V':18,
        'W':19,
        'Y':20,   
    }
    #return switcher.get(a, "nothing")
    return switcher.get(a,0)

In [ ]:
# Open the file with read o permit
data=open('seqPF14.txt', 'r')
# use readlines to read all lines in the file
# The variable "seqs" is a list containing all lines
seqs = data.readlines()
# close the file after reading the lines.
data.close()

In [ ]:
M=np.size(seqs)
L=len(seqs[0])-1
Np=int(L*(L-1)/2)
align=np.zeros((M,L)).astype(int)
#Convert in a number
for m in range (M):
    for i in range (L):
        align[m,i]=letter2number(seqs[m][i])

In [ ]:
#Expand the alignment in a bynary (M,20xL) array
#gauge last a.a. remove the last symbol
q=20
msa20=np.zeros((M,L*q)).astype(int)
for i in range(L):
    for a in range (q):
        msa20[:,i*q+a]=(align[:,i]==a)

In [ ]:
# Couplings in zero sum gauge and Frobenius Norm
def Frobenius(J):
    F=np.zeros((L,L))
    Fv=np.zeros(Np)
    ind=np.zeros((Np,2)).astype(int)
    indv=np.zeros((L,L)).astype(int)
    l=0
    for i in range (L):
        for j in range(i+1,L):
            #matrix  21x21 including also the gauge symbol
            jinf2=np.zeros((q+1,q+1))
            jinf2[0:q,0:q]=J[i*q:(i+1)*q,j*q:(j+1)*q]
            #zero-sum matrix
            #J_norm=np.transpose(np.transpose(jinf2 - np.mean(jinf2,0))
            #                     - np.mean(jinf2,1)) + np.mean(jinf2)
            J_norm=jinf2 - np.mean(jinf2,0)
            J_norm=J_norm - np.mean(J_norm,1,keepdims=True)
            #Frobenius norm
            F[i,j] = LA.norm(J_norm)
            F[j,i]=F[i,j]
            ind[l,0]=i
            ind[l,1]=j
            Fv[l]=F[i,j]
            indv[i,j]=l
            l=l+1        
    return [Fv,indv,F,ind]

In [ ]:
def Apc(F):
    #Average Product Correction which improves contact Predictions
    Fapc=np.zeros(Np)
    avcoupl1=np.sum(F,1)/L
    sumj=np.sum(avcoupl1)/L
    l=0
    for i in range (L):
        for j in range (i+1,L):
            Fapc[l]=F[i,j]-avcoupl1[i]*avcoupl1[j]/sumj
            l=l+1         
    return Fapc           

In [ ]:
def ppv(Finput,indv,ind):
    #Sort Finput in reverse order 
    Fsort=np.sort(Finput)[::-1]
    Fsort_index=np.argsort(Finput)[::-1]

    #Read structural Distances
    backmap_distances=np.loadtxt('Data/backmap_distances_e_PF00014.txt')
    distv=np.ones(Np)*50;
    lb=np.size(backmap_distances,0)
    dist=np.zeros((L,L))
    for l in range (lb):
        i=int(backmap_distances[l,0]-1)
        j=int(backmap_distances[l,1]-1)
        dist[i,j]=backmap_distances[l,4]
        leff=indv[i,j]
        distv[leff]=backmap_distances[l,4]

    #Positive Predicted Value for all contacts 
    sux=np.zeros(Np+1)
    suxn=np.zeros(Np+1)
    dc2=8;
    sux[0]=0
    for i in range (Np):
        if distv[Fsort_index[i]]<dc2:
            sux[i+1]=sux[i]+1
        else:
            sux[i+1]=sux[i]
        suxn[i+1]=sux[i+1]/(i+1)

    #Positive Predicted Value for contacts distant along the backbone i-j>4
    sud=np.zeros(Np+1)
    sudn=np.zeros(Np+1)
    dc2=8
    sud[0]=0
    ni=0
    for i in range (Np):
        if ind[Fsort_index[i],1]-ind[Fsort_index[i],0] >4:
            if distv[Fsort_index[i]]<dc2:
                        sud[ni+1]=sud[ni]+1
            else:
                        sud[ni+1]=sud[ni]  
            sudn[ni+1]=sud[ni+1]/(ni+1)
            ni=ni+1
    return [suxn,sudn]
    